# Introduction
## Goals
By the end of this course, you should be able to
- Do basic data analysis using R or Python/Pandas, with a special emphasis on
  - triton, or other similar HPC cluster environments
  - workflows, I/O strategies etc. that work on HPC clusters.

What this course is NOT:
- A basic course in programming. We don't expect you to have prior knowledge of R or Python, but some programming experience is required.
- A basic course in statistics / machine learning. As part of the course we'll do some simple stuff, but we expect that you either understand the statistics from before or learn it on your own.

Topics that we're going to cover
- The dataframe data structure, and how it relates to other common data structures.
- Working with dataframes. Indexing, etc.
- Visualizing your results.

# Data structures and data frames
What is a data frame? Lets start by comparing to the other usual data structures you might have come across:

## Scalar
A scalar variable is just a single value

In [ ]:
a = 42

A variable has a type. Python has the builtin function type() that gives you the type of an object:

In [ ]:
type(a)

If we create another variable, say, a string, we see that it has a different type

In [ ]:
b = "hello"
type(b)

## Containers
A container is a collection of values. Various types of containers exist, differing in how the different values are stored. This produces different performance and storage efficiency semantics. That is, depending on what kind of operations you want to do on your collection of values, you choose a different kind of container.
### Lists
A list is a sequential array of values. Note that each value can be of a different type. Also the type of the list does not depend on the type of the contained values

In [ ]:
a_list = [1, "hello"]
type(a_list)

In [ ]:
type(a_list[0])

You can add stuff to a list after you have initially created it:

In [ ]:
a_list.append(1.2)
a_list

### Dictionaries
A dictionary is an unordered collection of key-value pairs. You can quickly look up a value by providing the key. E.g. a phone book:

In [ ]:
phonebook = {"Janne":123, "Richard":456}

In [ ]:
phonebook

In [ ]:
phonebook["Janne"]

If you have experience with other programming languages, you might know dictionaries as "associative arrays", "hash tables", or "maps".
### Numpy arrays
Numpy fulfills the need of the numerical computing community for an efficient data structure for dense multi-dimensional arrays:

In [ ]:
import numpy as np
n = np.array((1, 2, 3))
n2 = np.array(((1, 2, 3), (4, 5, 6)))
n2

You can see the shape of a numpy array with the shape attribute:

In [ ]:
n2.shape

Contrary to a list, each value in a numpy array must be of the same type. You can see the type of the values in a numpy array from the dtype attribute:

In [ ]:
n2.dtype

In [ ]:
n[0] = 4
n

In [ ]:
n[0] = "hello"

Why this restriction? It comes down to the "efficient" word above. Since a list can have elements of arbitrary type, it needs an extra layer of indirection:

![a_list in memory](img/a_list.svg)

And for a multidimensional array, it's even worse; each element is then a reference to a nested list etc.

In contrast, a numpy ndarray is stored densely in memory:

![ndarray in memory](img/ndarray.svg)

A multidimensional ndarray is stored in memory as a single one-dimensional data array, and the shape information stored in the metadata is used to calculate the correct element to access.

Numpy ndarrays are stored in the same way that arrays in C or Fortran are stored. This allows one to use battle-tested C/Fortran code working directly on ndarray data, all glued together with an easy to use Python layer. Essentially, this is what the entire numpy and scipy is mostly about.

## Data frames
So what is then a data frame? In short, it is a data structure for tabular data. Similar to a two-dimensional numpy ndarray, except that each column can be of a different type (in fact, currently in Pandas data frames are implemented similarly as a one-dimensional ndarray for each column). Data frames optionally have one column as an index, similar to e.g. RDBMS's, allowing quicker lookups of rows when using the index column.

An additional type of data supported by data frames is categorical data, or factors. These are useful when one wants to group a string column according to the string value. We'll get back to categorical data later. If you have used R, you'll know categorical data as factors.

Lets look at some simple examples:

In [ ]:
import pandas as pd

In [ ]:
d=pd.DataFrame({'name': ('foo', 'bar'), 'value': (1, 2)})
d

You see here that we have a dataframe with two rows and two columns. No wait, three columns! How come? The first column is the index column; a pandas dataframe always contains an index column. Often you want to use one of the columns in the dataset as the index, e.g. a timestamp or something like that. But if you don't specify an index column, it will create an extra column as the index column.

In [ ]:
d=pd.DataFrame({'name': ('foo', 'bar')}, index=(1,2))
d

We can give the index column a name with

In [ ]:
d.index.name = 'value'
d

Lets demonstrate some pandas features by loading a toy dataset to play around with. Pandas happens to ship with a few, here we use the famous iris dataset:

In [ ]:
import os.path
c=pd.read_csv(os.path.join(pd.__path__[0], 'tests/data/iris.csv'))

If the dataset is large, viewing the entire dataset on the screen is often impossible. Pandas has head and tail methods similar to the shell tools:

In [ ]:
c.head()

In [ ]:
c.tail(3) # Show the last 3 rows (default 5)

The datatypes of each column can be retrieved from the dtypes attribute:

In [ ]:
c.dtypes

The last column is of type 'object'. It looks like it's one of a few strings, indicating that it might be suitable as a categorical type. Lets see what happens if we try to make a categorical of it:

In [ ]:
pd.Categorical(c['Name'])

So there are only 3 different values the name can take. Lets replace that column in the dataset with the corresponding categorical:

In [ ]:
c['Name'] = pd.Categorical(c['Name'])

In [ ]:
c.dtypes

If you know that a column should be categorical upfront, you can specify that directly in read_csv:

In [ ]:
c=pd.read_csv(os.path.join(pd.__path__[0], 'tests/data/iris.csv'), dtype={'Name': 'category'})
c.dtypes

In addition to head() and tail() a useful way of quickly getting some idea about a dataset is the describe() method:

In [ ]:
c.describe()